In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import re
from tqdm import tqdm # 用於顯示進度條
import csv

# --- 1. 常數與提示詞定義 (來自您的 GRPO 腳本) ---

SYSTEM_PROMPT = """你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
請嚴格遵守以下作答格式：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結出最合理的編碼集合。  
</reasoning>  
<answer>  
僅輸出最終的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
"""

EXAMPLE_QUESTION = """
Sex: F
Service: MEDICINE
Allergies: Aspirin
Attending: ___
Chief Complaint: weakness, diarrhea
Major Surgical or Invasive Procedure: None

History of Present Illness:
Ms. ___ is a ___ year-old woman with PMH significant for chronic anemia, osteoporosis, hypertension, ataxia, and recent L5 fracture in the setting of recurrent falls who presents from home with fatigue and generalized weakness and diarrhea.  

The patient's recent history is notable for the follow:  
- On ___, she presented with 4 days of LBP s/p fall from standing at which time imaging revealed acute L5 fracture. She was evaluated by Spine team who recommended early mobilization, pain control, but no brace required. She was evaluated by ___, and discharged to ___.  
- She was discharged home with ___ on ___.  
- On ___, she again presented to ___ s/p fall from standing while trying to reach for a glass of water. She did have a occipital scalp hematoma, but imaging including ___, C-spine CT, and L hip X-ray were negative for acute process so patient was discharged home.  

She now represents with generalized fatigue and diarrhea. In the setting of opiates for her L5 fracture, the patient has had constipation (5 days with no BM) for which she took a "natural laxative" the evening prior to presentation. The patient had 2 bowel movements in the morning of presentation and one episode of incontinence with diarrhea while sleeping. In this setting, she felt very weak and called EMS and was brought to ___ ED.

What ICD-10 codes should be assigned? Please just tell me which codes in the end
"""
EXAMPLE_REASONING = """The patient has chronic anemia, documented and clinically relevant, which corresponds to D500.  
There is vitamin deficiency suggested, which fits E538.  
She also has visual impairment history, which maps to H548.  
Hypertension is a chronic condition, coded as I10.  
Diarrhea due to laxative use is coded as K521 (toxic gastroenteritis and colitis).  
Her recent fracture with underlying osteoporosis corresponds to M810.  
Ataxia is a chronic neurologic condition, coded as R270.  
There is adverse effect of drugs (laxatives/opioids), so T474X5A applies.  
Environmental factor of injury is Y92099 (unspecified place of occurrence).  
She also has aspirin allergy, captured by Z9181.  

Therefore, the most appropriate ICD-10 codes are:
</reasoning>
"""

EXAMPLE_ANSWER = "D500 E538 H548 I10 K521 M810 R270 T474X5A Y92099 Z9181"

/mnt/data2/icd10/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)



def extract_xml_answer(text: str) -> str:
    """提取 <answer> 標籤內的內容"""
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    answer = answer.strip()
    return answer


def is_text(text: str) -> bool:
    """判斷輸入字串是否主要為文字敘述（而非 ICD code 等短編碼）"""
    letters_and_spaces = sum(c.isalpha() or c.isspace() for c in text)
    ratio = letters_and_spaces / max(1, len(text))  
    return ratio > 0.5


def compute_f1(pred: str, true: str) -> float:
    """
    計算 F1 分數 (前三個字符匹配即算正確)
    
    Args:
        pred: 預測的 ICD 編碼字串,例如 "A010 B021"
        true: 真實的 ICD 編碼字串,例如 "A010 C032"
    
    Returns:
        F1 分數 (0.0 到 1.0)
    """
    # 將字串轉換為 set
    pred = pred.replace('.', '')
    true = true.replace('.', '')
    
    def parse_codes(text):
        if ',' in text:
            return set(code.strip().upper() for code in text.split(',') if code.strip())
        else:
            return set(code.strip().upper() for code in text.split() if code.strip())
    
    pred_set = parse_codes(pred)
    true_set = parse_codes(true)
    
    # 如果都是空集合,視為完全匹配
    if not pred_set and not true_set:
        return 1.0
    
    # 轉換為前三個字符的集合
    def get_prefix_set(codes):
        return set(code[:3] for code in codes if len(code) >= 3)
    
    pred_prefix = get_prefix_set(pred_set)
    true_prefix = get_prefix_set(true_set)
    
    # 計算 TP, FP, FN
    tp = len(pred_prefix & true_prefix)
    fp = len(pred_prefix - true_prefix)
    fn = len(true_prefix - pred_prefix)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    if precision + recall == 0:
        return 0.0
    
    f1 = 2 * precision * recall / (precision + recall)
    print(f"✅ Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return f1



def count_xml(text: str) -> float:
    """
    檢查 XML 標籤格式的完整性
    
    Returns:
        分數 (0.0 到 0.5),分數越高表示格式越完整
    """
    score = 0.0
    
    if text.count("<reasoning>\n") == 1:
        score += 0.125
    if text.count("\n</reasoning>\n") == 1:
        score += 0.125
    if text.count("\n<answer>\n") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>\n")[-1])
        score -= extra_content * 0.001
    if text.count("\n</answer>") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>")[-1]) - 1
        score -= max(0, extra_content) * 0.001
    
    return max(0.0, score)  # ✅ 確保不會是負數

In [3]:
# --- 3. 載入模型與 Tokenizer ---
model_path = "./models/Qwen3-14B-sft" 

print(f"正在載入模型: {model_path} ...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
    trust_remote_code=True  # 關鍵修改
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("模型載入完畢。")

# --- 4. 載入並處理資料 ---

print("正在載入 note_icd_data.jsonl ...")
chat_style_data = []
try:
    with open("note_icd_data.jsonl", "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= 50:
                print(f"已讀取前 {i} 筆資料 (上限 10 筆)。")
                break
            
            item = json.loads(line)  
            
            chat_prompt = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": EXAMPLE_QUESTION},
                {"role": "assistant", "content": f"<reasoning>{EXAMPLE_REASONING}</reasoning><answer>{EXAMPLE_ANSWER}</answer>"},
                {"role": "user", "content": item["question"]}
            ]
            
            ground_truth_answer= extract_icd_answer(item["answer"])
            
            chat_style_data.append({
                "prompt": chat_prompt,
                "answer": ground_truth_answer,
                "raw_question": item["question"]
            })
except FileNotFoundError:
    print("錯誤：找不到 'note_icd_data.jsonl' 檔案。")
    exit()

print(f"資料載入完畢，共 {len(chat_style_data)} 筆。")

正在載入模型: ./models/Qwen3-14B-sft ...


Loading checkpoint shards: 100%|██████████| 8/8 [00:04<00:00,  1.70it/s]

模型載入完畢。
正在載入 note_icd_data.jsonl ...
已讀取前 50 筆資料 (上限 10 筆)。
資料載入完畢，共 50 筆。


In [4]:
# --- 5. 執行 Baseline 推論並計算所有指標 ---
# 用於 Baseline Micro-F1
all_predictions_baseline = []
all_ground_truths_baseline = []
all_acc_baseline = []
all_strict_baseline = []
all_completion_lengths = []

# === 新增：初始化 CSV 檔案,寫入表頭 ===
csv_filename = "test_qwen_14B_sft.csv"
with open(csv_filename, 'w', newline='', encoding='utf-8') as f:
    fieldnames = ['accuracy_reward', 'completion_length', 'ground_truths_ans', 'predictions_ans']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

print(f"📁 CSV 檔案已初始化: {csv_filename}\n")

with torch.no_grad():
    for idx, item in enumerate(tqdm(chat_style_data, desc="[Baseline] 執行推論與評估"), 1):
        chat_prompt = item["prompt"]
        gt_answer = item["answer"]
        
        try:
            prompt_str = tokenizer.apply_chat_template(
                chat_prompt, 
                tokenize=False, 
                add_generation_prompt=True
            )
        except Exception as e:
            # 手動回退格式 (適用於 MedGemma)
            print(f"手動回退格式: {e}")
            prompt_str = (
                f"<start_of_turn>system\n{chat_prompt[0]['content']}<end_of_turn>\n"
                f"<start_of_turn>user\n{chat_prompt[1]['content']}<end_of_turn>\n"
                f"<start_of_turn>model\n{chat_prompt[2]['content']}<end_of_turn>\n"
                f"<start_of_turn>user\n{chat_prompt[3]['content']}<end_of_turn>\n"
                f"<start_of_turn>model\n"
            )
        
        inputs = tokenizer(
            prompt_str, 
            return_tensors="pt", 
            max_length=4096,
            truncation=True
        ).to(model.device)
        
        outputs = model.generate(
            **inputs,
            max_new_tokens=2000,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
        
        input_length = inputs.input_ids.shape[1]
        generated_tokens = outputs[0][input_length:]
        assistant_reply = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        
        # === 印出完整資訊 ===
        print("\n" + "="*80)
        print(f"📝 樣本 #{idx}")
        print("="*80)
        print(f"\n🤖 模型完整回答:")
        print("-"*80)
        print(assistant_reply)
        print("-"*80)
        
        all_ground_truths_baseline.append(gt_answer)
        
        # 記錄 completion length (token 數)
        completion_length = len(generated_tokens)
        print(f"\n📊 完成的回覆長度 (token 數): {completion_length}")
        all_completion_lengths.append(completion_length)
        
        # 計算 Baseline metrics
        predicted_codes_baseline = extract_icd_answer(assistant_reply)
        print(f"🔍 提取的 ICD-10 編碼: {predicted_codes_baseline}")
        all_predictions_baseline.append(predicted_codes_baseline)

        print(f"\n🎯 Ground Truth Answer:")
        print(f"   {gt_answer}")
        
        acc_baseline = compute_f1(predicted_codes_baseline, gt_answer)
        print(f"✅ Baseline F1 分數: {acc_baseline:.4f}")
        all_acc_baseline.append(acc_baseline)
        
        
        with open(csv_filename, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writerow({
                'accuracy_reward': acc_baseline,
                'completion_length': completion_length,
                'ground_truths_ans': gt_answer,
                'predictions_ans': predicted_codes_baseline
            })
        
        print(f"💾 結果已儲存至 {csv_filename} (第 {idx} 筆)")
        print("="*80 + "\n")

print("\n" + "="*80)
print("🎉 推論完成!")
print("="*80)

# --- 6. 顯示最終統計 ---
if not all_predictions_baseline:
    print("沒有可評估的結果。")
else:
    print("\n" + "="*50)
    print("  Baseline (sft-4b-it) 最終統計")
    print("="*50)
    print(f"✅ 總共處理: {len(all_acc_baseline)} 筆資料")
    print(f"📊 平均 F1 分數: {sum(all_acc_baseline)/len(all_acc_baseline):.4f}")
    print(f"📏 平均回覆長度: {sum(all_completion_lengths)/len(all_completion_lengths):.1f} tokens")
    print(f"💾 結果已完整儲存至: {csv_filename}")
    print("="*50)

📁 CSV 檔案已初始化: test_qwen_14B_sft.csv



[Baseline] 執行推論與評估:   2%|▏         | 1/50 [00:25<21:00, 25.73s/it]


📝 樣本 #1

🤖 模型完整回答:
--------------------------------------------------------------------------------
<think>

<reasoning>Step-by-step reasoning:
1.  **Chronic Anemia (D500):** The patient's PMH explicitly states "chronic anemia," which is a long-standing condition.
2.  **Vitamin Deficiency (E538):** The patient's PMH includes "osteoporosis," and the HPI mentions "vitamin deficiency" as a significant part of her history. This code is used for unspecified vitamin deficiency.
3.  **Visual Impairment (H548):** The patient's PMH includes "visual impairment," which is a chronic condition.
4.  **Hypertension (I10):** The patient's PMH explicitly states "hypertension," a common chronic condition.
5.  **Diarrhea (K521):** The patient presents with "diarrhea" and "one episode of incontinence with diarrhea while sleeping." This is a direct symptom.
6.  **Osteoporosis (M810):** The patient's PMH explicitly states "osteoporosis," a chronic condition.
7.  **Ataxia (R270):** The patient's PMH explici

[Baseline] 執行推論與評估:   4%|▍         | 2/50 [01:28<38:04, 47.60s/it]


📝 樣本 #2

🤖 模型完整回答:
--------------------------------------------------------------------------------
<think>

<reasoning>Step-by-step reasoning:
1.  **Cardiogenic shock (Chief Complaint):** The patient's chief complaint is cardiogenic shock, which is a severe condition directly related to heart failure. This is the primary diagnosis and should be coded as I5021 (Acute on chronic systolic (congestive) heart failure with cardiogenic shock).
2.  **Ischemic cardiomyopathy with EF 25% (PMHx):** This is a chronic condition with a significant impact on the patient's health. The ejection fraction (EF) of 25% indicates severe systolic dysfunction. This is coded as I5022 (Chronic systolic (congestive) heart failure with reduced ejection fraction).
3.  **3 vessel CAD s/p RCA stenting (PMHx):** This is a chronic condition and a significant part of the patient's medical history. It is coded as I2510 (Atherosclerotic heart disease of native coronary artery without angina pectoris).
4.  **DMII (PMHx)

...


📝 樣本 #48

🤖 模型完整回答:
--------------------------------------------------------------------------------
<think>

<reasoning>Step-by-step reasoning:
1.  **Posterior fossa dural AVM (Chief Complaint and History of Present Illness):** The patient's chief complaint and HPI explicitly state "posterior fossa dural AVM" and "cerebral angiogram which confirmed diagnosis." This is a direct diagnosis of an arteriovenous malformation in the posterior fossa, which is a specific type of brain vascular malformation. The ICD-10 code for this is **I62.810** (Arteriovenous malformation of brain, unspecified, without hemorrhage).
2.  **Left hemifacial spasm (History of Present Illness):** The patient is "followed in the ___ clinic for left hemifacial spasm for about ___ year." This is a chronic neurological condition characterized by involuntary muscle contractions on one side of the face. The ICD-10 code for this is **G44.819** (Other specified disorders of sleep and wakefulness, unspecified).
3.  **Head

[Baseline] 執行推論與評估:  98%|█████████▊| 49/50 [35:51<00:31, 31.75s/it]


📝 樣本 #49

🤖 模型完整回答:
--------------------------------------------------------------------------------
<think>

<reasoning>Step-by-step reasoning:
1.  **Rheumatic heart disease with history of MVR and TVR:** The patient has a history of "rheumatic heart disease" and has undergone "MVR (mitral valve replacement) and TVR (tricuspid valve replacement)." This directly corresponds to the ICD-10 code **I09.81** (Rheumatic heart disease with prosthetic valve).
2.  **Fever:** The chief complaint is "Fevers," and the patient reports "___ days of atraumatic pain in his left knee, left wrist, and left elbow, along with fever to 101 at rehab and ___ measured today at 19." This indicates a fever, which is coded as **R50.9** (Fever, unspecified).
3.  **Arthralgia (pain in joints):** The patient reports "___ days of atraumatic pain in his left knee, left wrist, and left elbow." This is a clear description of pain in multiple joints, which is coded as **M25.519** (Pain in unspecified joint, left side).

[Baseline] 執行推論與評估: 100%|██████████| 50/50 [36:22<00:00, 43.65s/it]


📝 樣本 #50

🤖 模型完整回答:
--------------------------------------------------------------------------------
<think>

<reasoning>Step-by-step reasoning:
1.  **Chief Complaint and Major Symptoms:**
    *   "right sided tinnitus and sensory changes" directly points to **R270 (Ataxia)** for the sensory changes and **H919 (Other specified hearing disorders)** for the tinnitus.
    *   "right sided tingling and R ear tinnitus" further supports **R270 (Ataxia)** for the tingling and **H919 (Other specified hearing disorders)** for the tinnitus.
    *   "R hemibody paresthesias (face/arm/leg)" is a clear indication of **R270 (Ataxia)** for the sensory changes.
    *   "buzzing feeling in the R face, arm and leg" is a form of paresthesia, supporting **R270 (Ataxia)**.
    *   "numbness of the R face and R finger tips" is another form of sensory change, supporting **R270 (Ataxia)**.
    *   "pulsing" sound in the R ear is a specific type of tinnitus, supporting **H919 (Other specified hearing disorder